In [1]:
# pip install langchain_community

In [2]:
# pip install langchain_openai

In [4]:
# pip install Chroma

In [ ]:
# pip install chromadb

In [6]:
# pip install pymupdf

In [7]:
# pip install langchain-anthropic

In [8]:
import os

# Import classes from modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic

In [9]:
import json
from typing import overload, Union
from langchain_core.documents import Document #type: ignore

@overload
def extract_data(paths: Union[str, list[str]], main_key: str) -> list[Document]: ...

@overload
def extract_data(paths: Union[str, list[str]]) -> list[Document]: ...

def extract_data(paths: Union[str, list[str]], main_key: str = "items") -> list[Document]:
    documents: list[Document] = []
    
    if isinstance(paths, str):
        paths = [paths]
    
    for path in paths:
        with open(path, "r", encoding='utf8') as json_file:
            data: list[dict[str, str]] = json.load(json_file)[main_key]
            for each_data in data:
                page_content: str = ""
                for k in each_data:
                    page_content = page_content + f"{k} {each_data[k]}"
                current_document = Document(
                    page_content=page_content,
                )
                documents.append(current_document)
    
    return documents

In [10]:
# Generate API KEY from Claude and OpenAI website and define as a variable.
os.environ["ANTHROPIC_API_KEY"] = "YOUR_API_KEYS"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEYS"

In [11]:
pdf_paths = ["ข้อมูลยา 50 ชนิด.pdf"]
json_file_paths = ["fruit.json"]

In [12]:
# Load PDF data
pdf_data = []
for path in pdf_paths:
    loader = PyMuPDFLoader(file_path=path)
    pdf_data.extend(loader.load_and_split())

# Load JSON data
json_data = extract_data(json_file_paths)

# Combine PDF and JSON data
total_data = pdf_data + json_data

In [13]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(total_data)

In [14]:
texts

[Document(metadata={'source': 'ข้อมูลยา 50 ชนิด.pdf', 'file_path': 'ข้อมูลยา 50 ชนิด.pdf', 'page': 0, 'total_pages': 9, 'format': 'PDF 1.4', 'title': 'ข้อมูลยา 50 ชนิด', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Skia/PDF m128 Google Docs Renderer', 'creationDate': '', 'modDate': '', 'trapped': ''}, page_content='นี่คือข้\nอมูลยาสามัญประจำบ้\nาน50 รายการพร้\nอมด้\nวยข้\nอมูลชื่อยา, ส่วนผสม, สรรพคุณ, ขนาดและวิธีใช้\n, คำ\nเตือน, และการเก็บรักษา:\n1. พาราเซตามอล(Paracetamol)\n- ส่วนผสม: พาราเซตามอล\n- สรรพคุณ: บรรเทาอาการปวดและลดไข้\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่500-1000 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน4 กรัมต่อวัน), เด็กตามคำแนะนำของ\nแพทย์\n- คำเตือน: ห้\nามใช้\nเกินขนาดที่กำหนด, ระวังการใช้\nร่วมกับยาที่มีพาราเซตามอล\n- การเก็บรักษา: เก็บในที่แห้\nงและเย็น, ห่างจากแสงแดด\n2. ไอบูโพรเฟน(Ibuprofen)\n- ส่วนผสม: ไอบูโพรเฟน\n- สรรพคุณ: บรรเทาอาการปวด, ลดไข้\n, และลดการอักเสบ\n- ขนาดและวิธีใช้\n: ผู้\nใหญ่200-400 มิลลิกรัมทุก4-6 ชั่วโมง(ไม่เกิน1.2 กรัมต่อวัน), เ

In [15]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [16]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai language model assistant.
    Answer the question based ONLY on the following context.


    {context}


    Original question: {question}""",
)

In [17]:
# Create the language model
llm1 = ChatAnthropic(model="claude-3-sonnet-20240229", max_tokens=1024)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [18]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

User:  ยาแกไอ ต้องกินยังไง


Chatbot: ข้อมูลที่ให้มานั้นเป็นรายละเอียดเกี่ยวกับผลไม้ชนิดต่างๆ ไม่มีข้อมูลเกี่ยวกับยาแก้ไอหรือวิธีการกินยาแก้ไอแต่อย่างใด ดังนั้นจากบริบทดังกล่าว ผมไม่สามารถตอบคำถามเกี่ยวกับวิธีการกินยาแก้ไอได้


User:  ยาแก้ไอ มีสรรพคุณยังไง


Chatbot: ยาแก้ไอไม่ได้ถูกกล่าวถึงในข้อมูลที่ให้มา ข้อมูลนี้ระบุเฉพาะยาสามัญประจำบ้าน 5 ชนิดคือ พาราเซตามอล, ไอบูโพรเฟน, ยาลดกรดอะลูมิเนียม-แมกนีเซียม, ยาคลอเฟนิรามีน และยาอะม็อกซีซิลลิน ไม่มีข้อมูลเกี่ยวกับสรรพคุณของยาแก้ไอ


User:  ผลไม้ชนิดไหนมีรสชาติ เปี้ยวบ้าง


Chatbot: จากข้อมูลที่ให้มา ผลไม้ที่มีรสชาติเปรี้ยวได้แก่
1. มะนาว 
2. กีวี่


User:  มีอีกไหม


Chatbot: ใช่ มีผลไม้อื่นๆ ที่มีรสชาติเปรี้ยวด้วย ได้แก่ 
- มะนาว มีรสชาติเปรี้ยว
- กีวี่ มีรสชาติเปรี้ยวอมหวาน
- สตรอว์เบอร์รี มีรสชาติเปรี้ยวอมหวาน


User:  q
